### 1. Documentación de haystackAI
https://docs.haystack.deepset.ai/docs/components_overview

In [1]:
import logging
# logging.basicConfig(level=logging.DEBUG)
# logging.basicConfig(level=logging.INFO)
logging.basicConfig(level=logging.WARNING)

#### Demo seleccionando el modelo

In [2]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.utils import Secret

# Definición del template para construir prompts
prompt_template = """
    Given these documents, answer the question.\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \nQuestion: {{query}}
    \nAnswer:
    """

# Crear pipeline
p = Pipeline()

# Agregar componentes
p.add_component(instance=InMemoryBM25Retriever(document_store=InMemoryDocumentStore()), name="retriever")
p.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")

# Configuración del generador para usar un modelo específico
p.add_component(
    instance=OpenAIGenerator(
        api_key=Secret.from_env_var("OPENAI_API_KEY"),  # Clave API desde variable de entorno
        model="gpt-4o-mini-2024-07-18"  # Modelo específico
    ),
    name="llm"
)

p.add_component(instance=AnswerBuilder(), name="answer_builder")

# Conectar componentes del pipeline
p.connect("retriever", "prompt_builder.documents")
p.connect("prompt_builder", "llm")
p.connect("llm.replies", "answer_builder.replies")
p.connect("llm.meta", "answer_builder.meta")
p.connect("retriever", "answer_builder.documents")

# Definir consulta
query = "Quién es rafael lopez aliaga alias porky?"

# Ejecutar el pipeline
result = p.run(
    {
        "retriever": {"query": query},
        "prompt_builder": {"query": query},
        "answer_builder": {"query": query},
    }
)

# Imprimir el resultado
print(result)


{'answer_builder': {'answers': [GeneratedAnswer(data='Rafael López Aliaga, conocido como "Porky", es un político y empresario peruano. Nació el 5 de enero de 1967 en Lima, Perú. Es conocido por ser el alcalde de Lima, cargo al que accedió en las elecciones municipales de 2022. López Aliaga se ha destacado por su postura conservadora y sus propuestas relacionadas con la economía y la seguridad. Además, ha sido una figura controversial en la política peruana, siendo objeto de críticas y elogios por su estilo de liderazgo y sus declaraciones públicas.', query='Quién es rafael lopez aliaga alias porky?', documents=[], meta={'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 113, 'prompt_tokens': 40, 'total_tokens': 153, 'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_toke

#### Demo seleccionando el modelo por defecto

In [3]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.utils import Secret

prompt_template = """
    Given these documents, answer the question.\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \nQuestion: {{query}}
    \nAnswer:
    """

p = Pipeline()
p.add_component(instance=InMemoryBM25Retriever(document_store=InMemoryDocumentStore()), name="retriever")
p.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
p.add_component(instance=OpenAIGenerator(api_key=Secret.from_env_var("OPENAI_API_KEY")), name="llm")
p.add_component(instance=AnswerBuilder(), name="answer_builder")
p.connect("retriever", "prompt_builder.documents")
p.connect("prompt_builder", "llm")
p.connect("llm.replies", "answer_builder.replies")
p.connect("llm.meta", "answer_builder.meta")
p.connect("retriever", "answer_builder.documents")
query = "What is the capital of France?"
result = p.run(
    {
        "retriever": {"query": query},
        "prompt_builder": {"query": query},
        "answer_builder": {"query": query},
    }
)

print(result)

{'answer_builder': {'answers': [GeneratedAnswer(data='The capital of France is Paris.', query='What is the capital of France?', documents=[], meta={'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 7, 'prompt_tokens': 34, 'total_tokens': 41, 'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0)}})]}}


#### Demo seleccionando con descripcion

In [4]:
# Importación de componentes clave desde Haystack
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.utils import Secret

# Definición de un template para construir prompts que serán enviados al modelo de lenguaje
# Este prompt toma documentos y una pregunta como entrada y solicita una respuesta.
prompt_template = """
    Given these documents, answer the question.\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \nQuestion: {{query}}
    \nAnswer:
    """

# Creación de un pipeline para conectar diferentes componentes
p = Pipeline()

# Agregando un componente de recuperación de documentos basado en BM25
# Este componente buscará documentos relevantes en un almacén en memoria.
p.add_component(instance=InMemoryBM25Retriever(document_store=InMemoryDocumentStore()), name="retriever")

# Agregando un PromptBuilder, que genera el prompt basado en el template y los documentos recuperados
p.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")

# Agregando un generador (modelo de lenguaje) que utiliza OpenAI
# La clave API se carga desde una variable de entorno utilizando Secret.from_env_var.
p.add_component(instance=OpenAIGenerator(api_key=Secret.from_env_var("OPENAI_API_KEY")), name="llm")

# Agregando un AnswerBuilder, que toma las respuestas generadas por el modelo
# y las formatea en objetos GeneratedAnswer enriquecidos con datos como los documentos referenciados.
p.add_component(instance=AnswerBuilder(), name="answer_builder")

# Conectando los componentes del pipeline para definir el flujo de datos.

# Conexión del recuperador con el PromptBuilder: los documentos relevantes alimentan el prompt.
p.connect("retriever", "prompt_builder.documents")

# Conexión del PromptBuilder con el generador: el prompt generado se envía al modelo de lenguaje.
p.connect("prompt_builder", "llm")

# Conexión de las respuestas del generador al AnswerBuilder: las respuestas crudas son procesadas y formateadas.
p.connect("llm.replies", "answer_builder.replies")

# Conexión de los metadatos generados por el modelo al AnswerBuilder: los metadatos enriquecen los objetos de respuesta.
p.connect("llm.meta", "answer_builder.meta")

# Conexión de los documentos recuperados directamente al AnswerBuilder: los documentos ayudan a contextualizar las respuestas.
p.connect("retriever", "answer_builder.documents")

# Definición de la consulta que se va a procesar
query = "What is the capital of France?"

# Ejecución del pipeline con la consulta, pasando datos necesarios a cada componente
result = p.run(
    {
        "retriever": {"query": query},            # La consulta para recuperar documentos.
        "prompt_builder": {"query": query},      # La consulta para construir el prompt.
        "answer_builder": {"query": query},      # La consulta para contextualizar las respuestas.
    }
)

# Imprime el resultado final generado por el pipeline
print(result)


{'answer_builder': {'answers': [GeneratedAnswer(data='The capital of France is Paris.', query='What is the capital of France?', documents=[], meta={'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 7, 'prompt_tokens': 34, 'total_tokens': 41, 'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0)}})]}}


#### Resultados

In [6]:
response_data = result['answer_builder']['answers'][0].data
print(response_data) 

The capital of France is Paris.


In [7]:
query = result['answer_builder']['answers'][0].query
print(query)

What is the capital of France?


In [8]:
documents = result['answer_builder']['answers'][0].documents
print(documents)

[]


In [9]:
meta = result['answer_builder']['answers'][0].meta
print(meta)

{'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 7, 'prompt_tokens': 34, 'total_tokens': 41, 'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0)}}


In [10]:
model_used = meta['model']
print(model_used)

gpt-4o-mini-2024-07-18


In [11]:
total_tokens = meta['usage']['total_tokens']
print(total_tokens)

41


In [12]:
for answer in result['answer_builder']['answers']:
    print(f"Response: {answer.data}")
    print(f"Query: {answer.query}")
    print(f"Model: {answer.meta['model']}")

Response: The capital of France is Paris.
Query: What is the capital of France?
Model: gpt-4o-mini-2024-07-18
